In [3]:
import keras
import numpy as np
from keras import layers
import random
import sys
import io

path = './AllLyrics.txt'
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()
print('Corpus length:', len(text))


# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)


def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.8]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(500):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()


Corpus length: 164265
Number of sequences: 54735
Unique characters: 52
Vectorization...
epoch 1
Epoch 1/1
54735/54735 [==============================] - 165s 3ms/step - loss: 2.2698
--- Generating with seed: " it helps you sleepand hold me tightdon't let me breathefeel"
------ temperature: 0.8
 it helps you sleepand hold me tightdon't let me breathefeely arlf in the you kers sheer sray i'm folos wres i'm ne fing it never your it it freme thound when you sheart gromy aro ghat homa know she tid darde my your hore dould erk ak you noon ays anding then if the meh rithe make shelm like then bad a sheen shearen i siy weeld for geentyou buen oummyou just it what noway just andshefun a'll seen that a showe bick, in rome ar lade then a gaand far ay her and work you and fllone in ard than and hern the like to shith just like the boren ay your broing oh 
epoch 2
Epoch 1/1
54735/54735 [==============================] - 178s 3ms/step - loss: 1.9008
--- Generating with seed: "never been brokethough 

t even know me at allbut i was made for loving youi was made a strangerso need to brokened you are my home, and the got for moor, now, let'ceme on the ends be, blaysgle of feeland nowand we'll never get nonkand in my bas and the conut every dide again, but i callred to sidedesing me tonyso where the manei see your brasting miss you were up a friends fine so that leaves and you'can chish) ninanow look me now what i know homei with a pranamiles's so in love for the lights come tooly.it never see like you, i'm never get say just life i know the walking come
epoch 12
Epoch 1/1
54735/54735 [==============================] - 138s 3ms/step - loss: 1.1643
--- Generating with seed: " my motherand i'm going there no more to roami'm only going "
------ temperature: 0.8
 my motherand i'm going there no more to roami'm only going i know my only.even, i wanna one to my eyes roaway growi remornand i'm nenell piokey and i'll fee your not in the she and trow and way, we can tall in a friends the fall t

C:\ProgramData\Anaconda3\envs\mykeras\lib\site-packages\ipykernel_launcher.py:56: RuntimeWarning: divide by zero encountered in log


ts are crazil that's way.sell me gonna beat thes words you should lived in the tood on the conlonyour on the scarses with you.i would look you one, to hating to see my paine as pliaskeep in the dreams on the outh the got to s,put me getting bolit's beends i
epoch 18
Epoch 1/1
54735/54735 [==============================] - 169s 3ms/step - loss: 1.0973
--- Generating with seed: "our ripped jeansholding me close until our eyes meetyou won'"
------ temperature: 0.8
our ripped jeansholding me close until our eyes meetyou won't dina dayand i'll staywalk,i'm home that i hear that it's like the grad on the readsto and my erlienone with you were tears too all over me took and the could be the strumming me, and was about the been when .herd don't leeve in love alood and your eyes are alone, and i don't even scong, you're living like love and way out and then they stay the might me with a hear in that i take me to be the onelowsy now that i was gonna chan they rast's home on ming she only my momi

long nights, strange menand they sayshe's in the class a tears or why you said, you fall feel falling seck, and i haven't never stay, i'll see your handall time to pled to say and sawn was erting borns when i've got i ploke you were in love arenctcause i all too love me babeah love mei salting the ground upi di am rier townr)wo nooma dadd let me perflya hed to youendoy is moshon you were just base to be sime like you all over my heart stonesi love me like you are the seess and playing the streets high,s aloods inbout new on a longureand the con't baby, i
epoch 29
Epoch 1/1
54735/54735 [==============================] - 147s 3ms/step - loss: 0.9805
--- Generating with seed: " not a homeless life for meit's just i'm home less than i'd "
------ temperature: 0.8
 not a homeless life for meit's just i'm home less than i'd ly to see myderiti's stay by all my bocklike it rain like you say in the light is to a tanneor can list carsand all if i went to take the been wart face to bebut sime on a

e you better nownowi'm gonna paint you by numbersand colour way to died as and there's arrings life like, all my bastmege on and i'll take my hands up a prame ?leave me goodoh what i'll take it wasne, one heart and for my trady one..... you down to let me but let regong only go homehandbeen a song, don't wanna go beliffll halfelif the screambery dver to the maan there's not the shoulders not every dit scarting blonft things and i'm gonna descrositnow ress comes,i ave pend of a foll baing to be someand all things rightbut i can wanna be lovedmomentwild al
epoch 39
Epoch 1/1
54735/54735 [==============================] - 129s 2ms/step - loss: 0.8952
--- Generating with seed: "y, that i'm the only onebut i know that god made another one"
------ temperature: 0.8
y, that i'm the only onebut i know that god made another one mexthen the nume sees arourd, your heart a piate through i'll say i i miss you, we can go another mace to watch drunk at the tool in, was new a strongerleft like i was ha

where you wantsay the word and i'll take yabut i'd rather start for my bothto frlewith my per's wrotedocam with your upwan you take me back, warm ere and and one toing for time, won't you dreambery life and whyou love me ifm sightlifs novering and madei save burn and lettreep in your dadgally goingand if you're the only granndyou knew sen your dream and flow with you got to say it all the same as are, beforey my peorot of my bluegs my raa,mbut the fuckso deatsi don't know what i dos all this passe a liotein't never donna madeif you're not cause can travi
epoch 49
Epoch 1/1
54735/54735 [==============================] - 126s 2ms/step - loss: 0.8378
--- Generating with seed: "differentlywhen youfre with him i know youfre lonelyplease, "
------ temperature: 0.8
differentlywhen youfre with him i know youfre lonelyplease, onle band fall in love an the same for the laiethes i start with a heaven, i was just baid parning my eyesyour eyesfor i right sweet same like you shoomigann, i'm not wrog

ne's gotta win worst-dressedtaking my first steps into the sunlie.tay to sonesi never know i was never tall offmorurnembur for there is not the picture, and i'll was in your drink again tice.dive like my sluest, i'm still growing upround than tear the lime, and my bones, i'll be in love with younboby with me, then i'm still growing up, and i never want over endsi needay that cimple a, was now it's a .and i've never meant sweet wn was a dark bronet reallyfs loneor night.when you're the mad you'll cannay girend make me sit on the stay owhoon, i'll take yat
epoch 59
Epoch 1/1
54735/54735 [==============================] - 127s 2ms/step - loss: 0.7963
--- Generating with seed: "ple fall in love in mysterious waysmaybe it's all part of a "
------ temperature: 0.8
ple fall in love in mysterious waysmaybe it's all part of a perfeal never gland, now the paining things to say itssing to a tame grann.i made, you down, dcark in one, and she's not the time to youy night on people i cut here is my 